In [1]:
!pip install pydot --quiet
!pip install gensim --quiet
!pip install tensorflow-datasets --quiet
!pip install -U tensorflow-text --quiet
!pip install transformers --quiet
!pip install jsonlines --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 83.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.5 MB/s eta 0:00:00


In [2]:
from transformers import (
    BertTokenizer,
    BertModel,
    TFBertModel,
    BertConfig,
    AutoTokenizer,
    AutoModel,
)

In [3]:
import pandas as pd
import numpy as np

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
DATA_PATH = '/content/drive/MyDrive/w266_final_project/data/EmoCause'

In [7]:
DATA_PATH_LOCAL = './data/emocause'

In [7]:
# Dataset doesn't provide an explicit "train" set, we can attempt to partition the existing data
df_valid = pd.read_json(f'{DATA_PATH}/valid.json')
df_test = pd.read_json(f'{DATA_PATH}/test.json')

In [8]:
len(df_valid)

3775

In [9]:
len(df_test)

838

In [10]:
df_valid

,original_situation,tokenized_situation,emotion,conv_id,annotation,labels
0,I felt betrayed when my girlfriend kissed anot...,"[I, felt, betrayed, when, my, girlfriend, kiss...",__disappointed__,hit:4449_conv:8898,"[[girlfriend, 5], [kissed, 6], [another, 7], [...","[girlfriend, kissed, another, guy]"
1,I found letters that my wife wrote me when we ...,"[I, found, letters, that, my, wife, wrote, me,...",__sentimental__,hit:4725_conv:9451,"[[letters, 2], [wife, 5], [wrote, 6], [first, ...","[letters, wife, wrote, first, dating]"
2,I have a job interview next week that I have b...,"[I, have, a, job, interview, next, week, that,...",__prepared__,hit:4678_conv:9356,"[[job, 3], [interview, 4], [getting, 11], [rea...","[job, interview, getting, ready]"
3,I'm getting cold feet about moving to another ...,"[I, 'm, getting, cold, feet, about, moving, to...",__apprehensive__,hit:4740_conv:9481,"[[moving, 6], [another, 8], [city, 9]]","[moving, another, city]"
4,I feel so bad that I have to work so much and ...,"[I, feel, so, bad, that, I, have, to, work, so...",__guilty__,hit:4652_conv:9304,"[[work, 8], [ca, 12], [n't, 13], [play, 14], [...","[work, ca, n't, play, grandchild]"
...,...,...,...,...,...,...
3770,"when my Grandparents died, I was extremely ups...","[when, my, Grandparents, died, ,, I, was, extr...",__devastated__,hit:10796_conv:21593,"[[died, 3]]",[died]
3771,One time in high school I sat in chocolate sau...,"[One, time, in, high, school, I, sat, in, choc...",__embarrassed__,hit:10909_conv:21818,"[[sauce, 9]]",[sauce]
3772,our company is firing people,"[our, company, is, firing, people]",__sad__,hit:10792_conv:21584,"[[firing, 3], [people, 4]]","[firing, people]"
3773,I'm trying to transition my dog into not sleep...,"[I, 'm, trying, to, transition, my, dog, into,...",__sad__,hit:11321_conv:22643,"[[scratching, 25], [door, 28]]","[scratching, door]"


In [4]:
cause_model_name = 'mrm8488/spanbert-finetuned-squadv2'

spanbert_tokenizer = BertTokenizer.from_pretrained(cause_model_name)
spanbert_model = TFBertModel.from_pretrained(cause_model_name)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['qa_outputs.bias', 'qa_outputs.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [12]:
df = pd.concat([df_valid, df_test])

In [13]:
df

,original_situation,tokenized_situation,emotion,conv_id,annotation,labels
0,I felt betrayed when my girlfriend kissed anot...,"[I, felt, betrayed, when, my, girlfriend, kiss...",__disappointed__,hit:4449_conv:8898,"[[girlfriend, 5], [kissed, 6], [another, 7], [...","[girlfriend, kissed, another, guy]"
1,I found letters that my wife wrote me when we ...,"[I, found, letters, that, my, wife, wrote, me,...",__sentimental__,hit:4725_conv:9451,"[[letters, 2], [wife, 5], [wrote, 6], [first, ...","[letters, wife, wrote, first, dating]"
2,I have a job interview next week that I have b...,"[I, have, a, job, interview, next, week, that,...",__prepared__,hit:4678_conv:9356,"[[job, 3], [interview, 4], [getting, 11], [rea...","[job, interview, getting, ready]"
3,I'm getting cold feet about moving to another ...,"[I, 'm, getting, cold, feet, about, moving, to...",__apprehensive__,hit:4740_conv:9481,"[[moving, 6], [another, 8], [city, 9]]","[moving, another, city]"
4,I feel so bad that I have to work so much and ...,"[I, feel, so, bad, that, I, have, to, work, so...",__guilty__,hit:4652_conv:9304,"[[work, 8], [ca, 12], [n't, 13], [play, 14], [...","[work, ca, n't, play, grandchild]"
...,...,...,...,...,...,...
833,I saw a homeless guy the other day on my way h...,"[I, saw, a, homeless, guy, the, other, day, on...",__grateful__,hit:2046_conv:4092,"[[saw, 1], [homeless, 3], [guy, 4]]","[saw, homeless, guy]"
834,I was surprised to learn how different my two ...,"[I, was, surprised, to, learn, how, different,...",__surprised__,hit:195_conv:391,"[[learn, 4], [different, 6], [two, 8], [kids, ...","[learn, different, two, kids, personalities]"
835,"I wnet home, and my wife surpises me with a pi...","[I, wnet, home, ,, and, my, wife, surpises, me...",__joyful__,hit:352_conv:705,"[[wife, 6], [surpises, 7], [picture, 11], [fut...","[wife, surpises, picture, future, baby]"
836,I shouted to my mom,"[I, shouted, to, my, mom]",__guilty__,hit:922_conv:1845,"[[shouted, 1], [mom, 4]]","[shouted, mom]"


In [14]:
def extract_cause_span(df):
  cause_span = []

  for i in range(len(df)):
    starting_index = list(df.annotation)[i][0][1]
    ending_index = list(df.annotation)[i][-1][1]

    cause_span.append(list(df.tokenized_situation)[i][starting_index:ending_index+1])

  return cause_span

In [15]:
cause_span = extract_cause_span(df)

In [16]:
def join_cause_spans(cause_span):
  cause_span_joined = []

  for l in cause_span:
    running_str = ''

    for i in range(len(l)):
      if "'" in l[i] or l[i] == 'nt' or l[i] == '/' or l[i] == '-' or l[i] == '.' or l[i] == ',' or i == 0:
        running_str += l[i]
      elif l[i-1] == '/' or l[i-1] == '-':
        running_str += l[i]
      elif l[i] == ' ':
        pass
      else:
        running_str += ' ' + l[i]

    cause_span_joined.append(running_str)

  return cause_span_joined

In [17]:
cause_span_joined = join_cause_spans(cause_span)

In [18]:
df['cause_span'] = [s.lower() for s in cause_span_joined]

In [19]:
df_valid.emotion.value_counts()

__surprised__       187
__annoyed__         148
__sad__             140
__excited__         137
__impressed__       134
__sentimental__     128
__afraid__          124
__embarrassed__     124
__proud__           124
__disgusted__       121
__angry__           120
__confident__       120
__lonely__          120
__content__         120
__terrified__       119
__nostalgic__       118
__prepared__        118
__disappointed__    117
__jealous__         117
__anxious__         113
__furious__         112
__caring__          110
__anticipating__    110
__hopeful__         108
__grateful__        108
__joyful__          107
__ashamed__         102
__guilty__          101
__devastated__      101
__apprehensive__    100
__trusting__         98
__faithful__         69
Name: emotion, dtype: int64

Tokenization in this dataset does not match that with BERT. Have to pre-process and extract start and end indices according to BERT tokenization here as well.

In [20]:
# Convert to lower case to avoid discrepancies in BERT tokenization
df_proc = df
df_proc.original_situation = df_proc.original_situation.apply(str.lower)
df_proc = df_proc.reset_index(drop=True)
df_proc

,original_situation,tokenized_situation,emotion,conv_id,annotation,labels,cause_span
0,i felt betrayed when my girlfriend kissed anot...,"[I, felt, betrayed, when, my, girlfriend, kiss...",__disappointed__,hit:4449_conv:8898,"[[girlfriend, 5], [kissed, 6], [another, 7], [...","[girlfriend, kissed, another, guy]",girlfriend kissed another guy
1,i found letters that my wife wrote me when we ...,"[I, found, letters, that, my, wife, wrote, me,...",__sentimental__,hit:4725_conv:9451,"[[letters, 2], [wife, 5], [wrote, 6], [first, ...","[letters, wife, wrote, first, dating]",letters that my wife wrote me when we first da...
2,i have a job interview next week that i have b...,"[I, have, a, job, interview, next, week, that,...",__prepared__,hit:4678_conv:9356,"[[job, 3], [interview, 4], [getting, 11], [rea...","[job, interview, getting, ready]",job interview next week that i have been getti...
3,i'm getting cold feet about moving to another ...,"[I, 'm, getting, cold, feet, about, moving, to...",__apprehensive__,hit:4740_conv:9481,"[[moving, 6], [another, 8], [city, 9]]","[moving, another, city]",moving to another city
4,i feel so bad that i have to work so much and ...,"[I, feel, so, bad, that, I, have, to, work, so...",__guilty__,hit:4652_conv:9304,"[[work, 8], [ca, 12], [n't, 13], [play, 14], [...","[work, ca, n't, play, grandchild]",work so much and can't play with my grandchild
...,...,...,...,...,...,...,...
4608,i saw a homeless guy the other day on my way h...,"[I, saw, a, homeless, guy, the, other, day, on...",__grateful__,hit:2046_conv:4092,"[[saw, 1], [homeless, 3], [guy, 4]]","[saw, homeless, guy]",saw a homeless guy
4609,i was surprised to learn how different my two ...,"[I, was, surprised, to, learn, how, different,...",__surprised__,hit:195_conv:391,"[[learn, 4], [different, 6], [two, 8], [kids, ...","[learn, different, two, kids, personalities]",learn how different my two kids' personalities
4610,"i wnet home, and my wife surpises me with a pi...","[I, wnet, home, ,, and, my, wife, surpises, me...",__joyful__,hit:352_conv:705,"[[wife, 6], [surpises, 7], [picture, 11], [fut...","[wife, surpises, picture, future, baby]",wife surpises me with a picture of our future ...
4611,i shouted to my mom,"[I, shouted, to, my, mom]",__guilty__,hit:922_conv:1845,"[[shouted, 1], [mom, 4]]","[shouted, mom]",shouted to my mom


In [21]:
# Keep the dataframe rows for all indices that don't occur in the 'nones' list
nones_l = [121,
 547,
 682,
 1969,
 1998,
 2183,
 2502,
 2648,
 2694,
 2746,
 2796,
 2860,
 3235,
 3570,
 3600,
 3779,
 3803,
 3804,
 4121,
 4268,
 4472]
df_proper = df_proc.drop(nones_l)
df_proper = df_proper[['original_situation', 'emotion', 'cause_span']]
df_proper

,original_situation,emotion,cause_span
0,i felt betrayed when my girlfriend kissed anot...,__disappointed__,girlfriend kissed another guy
1,i found letters that my wife wrote me when we ...,__sentimental__,letters that my wife wrote me when we first da...
2,i have a job interview next week that i have b...,__prepared__,job interview next week that i have been getti...
3,i'm getting cold feet about moving to another ...,__apprehensive__,moving to another city
4,i feel so bad that i have to work so much and ...,__guilty__,work so much and can't play with my grandchild
...,...,...,...
4608,i saw a homeless guy the other day on my way h...,__grateful__,saw a homeless guy
4609,i was surprised to learn how different my two ...,__surprised__,learn how different my two kids' personalities
4610,"i wnet home, and my wife surpises me with a pi...",__joyful__,wife surpises me with a picture of our future ...
4611,i shouted to my mom,__guilty__,shouted to my mom


In [22]:
# Manually adjust cause spans (there weren't too many of these to pose a time concern)
df_improper = df_proc.iloc[nones_l]
df_improper = df_improper[['original_situation', 'emotion', 'cause_span']]
cause_improper_list = [
  "couldn't go to my friends party",
  'waiting it is the worst thing for me, if im waiting more than 5 minutes',
  'cannot agree on what to eat for dinner',
  "girlfriend's cat has cancer and we cannot afford the operation",
  "couldn't afford",
  'boss told me if i worked hard id get a raise',
  'forecast said itll be raining and cloudy today, instead its +30c and sunny',
  'cousin. her husband took her and their 4 kids to a county fair. ate about noon and headed home. they decided to stop at a cemetary to look around, lost track of time and now they are locked inside. it is 7pm and everyone is very hungry',
  'customer support is just so surprisingly stupid, theyre asking me for my game code',
  "didn't get approved for my dream car",
  'spilt wed wine on my moms red carpet',
  "neighbor would stay up all night playing lol. he'd be yelling into his headset at 4am",
  "neighbor's dog. i cannot stand the barking",
  "bunch of flower bushes this year. i cannot wait until next spring",
  "girlfriend have been dating for 3 years now. im going to ask her to marry me and i have no doubts",
  "didn't give a homeless man any money",
  "wasn't able to pay rent",
  "haven't been rollerblading since i was a little kid",
  "gets up at 3am for work. instead of preparing the night before she does everything in the am. i am so tired of being woken",
  "didn't fit into my old wedding dress",
  "grocery store today and had to wait in line for 15 minutes/ theres was only one cashier. i cant belive the store didnt have more employees working"
  ]
df_improper['cause_span'] = cause_improper_list
df_improper

,original_situation,emotion,cause_span
121,i feel guilty because i couldn't go to my frie...,__guilty__,couldn't go to my friends party
547,"i hate waiting it is the worst thing for me, i...",__anxious__,"waiting it is the worst thing for me, if im wa..."
682,my family cannot agree on what to eat for dinn...,__furious__,cannot agree on what to eat for dinner
1969,my girlfriend's cat has cancer and we cannot a...,__sad__,girlfriend's cat has cancer and we cannot affo...
1998,when i was a kid i really wanted to go to disn...,__disappointed__,couldn't afford
2183,my boss told me if i worked hard id get a raise,__faithful__,boss told me if i worked hard id get a raise
2502,im feeling a little bit disappointed right now...,__disappointed__,forecast said itll be raining and cloudy today...
2648,got a call from my cousin. her husband took he...,__angry__,cousin. her husband took her and their 4 kids ...
2694,i'm trying to recover a videogame account and ...,__surprised__,customer support is just so surprisingly stupi...
2746,"i didn't get approved for my dream car, it was...",__disappointed__,didn't get approved for my dream car


In [23]:
df_clean = pd.concat([df_proper,df_improper])
df_clean = df_clean.sort_index()
df_clean

,original_situation,emotion,cause_span
0,i felt betrayed when my girlfriend kissed anot...,__disappointed__,girlfriend kissed another guy
1,i found letters that my wife wrote me when we ...,__sentimental__,letters that my wife wrote me when we first da...
2,i have a job interview next week that i have b...,__prepared__,job interview next week that i have been getti...
3,i'm getting cold feet about moving to another ...,__apprehensive__,moving to another city
4,i feel so bad that i have to work so much and ...,__guilty__,work so much and can't play with my grandchild
...,...,...,...
4608,i saw a homeless guy the other day on my way h...,__grateful__,saw a homeless guy
4609,i was surprised to learn how different my two ...,__surprised__,learn how different my two kids' personalities
4610,"i wnet home, and my wife surpises me with a pi...",__joyful__,wife surpises me with a picture of our future ...
4611,i shouted to my mom,__guilty__,shouted to my mom


In [24]:
# Get tokenized situations.
sit_tokens = spanbert_tokenizer(list(df_clean.original_situation))['input_ids']

In [25]:
# Get tokenized cause spans. Remove start and end token.
cause_tokens = spanbert_tokenizer(list(df_clean['cause_span']))['input_ids']
cause_tokens = [c[1:-1] for c in cause_tokens]

In [26]:
def find_sub_list(sl,l):
    sll=len(sl)
    for ind in [i for i,e in enumerate(l) if e==sl[0]]:
        if l[ind:ind+sll]==sl:
            return (ind,ind+sll-1)

In [27]:
start_end_indices = []
for i in range(len(sit_tokens)):
  start_end_indices.append(find_sub_list(cause_tokens[i], sit_tokens[i]))

In [28]:
nones = [i for (i, e) in enumerate(start_end_indices) if e is None]

In [29]:
nones

[]

In [30]:
df_clean['start_index'] = [i[0] for i in start_end_indices]
df_clean['end_index'] = [i[1] for i in start_end_indices]

In [31]:
df_clean

,original_situation,emotion,cause_span,start_index,end_index
0,i felt betrayed when my girlfriend kissed anot...,__disappointed__,girlfriend kissed another guy,6,9
1,i found letters that my wife wrote me when we ...,__sentimental__,letters that my wife wrote me when we first da...,3,12
2,i have a job interview next week that i have b...,__prepared__,job interview next week that i have been getti...,4,13
3,i'm getting cold feet about moving to another ...,__apprehensive__,moving to another city,8,11
4,i feel so bad that i have to work so much and ...,__guilty__,work so much and can't play with my grandchild,9,20
...,...,...,...,...,...
4608,i saw a homeless guy the other day on my way h...,__grateful__,saw a homeless guy,2,5
4609,i was surprised to learn how different my two ...,__surprised__,learn how different my two kids' personalities,5,12
4610,"i wnet home, and my wife surpises me with a pi...",__joyful__,wife surpises me with a picture of our future ...,8,19
4611,i shouted to my mom,__guilty__,shouted to my mom,2,5


In [32]:
# Test string to check tokenization results
test_string = "letters that my wife wrote me when we first dating"
print(spanbert_tokenizer(test_string))
print(spanbert_tokenizer.tokenize(test_string))

{'input_ids': [101, 3784, 1115, 1139, 1676, 1724, 1143, 1165, 1195, 1148, 4676, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['letters', 'that', 'my', 'wife', 'wrote', 'me', 'when', 'we', 'first', 'dating']


In [33]:
# Generate labels for emotion classes
df_clean['emotion_label'] = df_clean.emotion.map({'__afraid__':0, '__angry__':1, '__annoyed__':2, '__anticipating__':3,
       '__anxious__':4, '__apprehensive__':5, '__ashamed__':6, '__caring__':7,
       '__confident__':8, '__content__':9, '__devastated__':10,
       '__disappointed__':11, '__disgusted__':12, '__embarrassed__':13,
       '__excited__':14, '__faithful__':15, '__furious__':16, '__grateful__':17,
       '__guilty__':18, '__hopeful__':19, '__impressed__':20, '__jealous__':21,
       '__joyful__':22, '__lonely__':23, '__nostalgic__':24, '__prepared__':25,
       '__proud__':26, '__sad__':27, '__sentimental__':28, '__surprised__':29,
       '__terrified__':30, '__trusting__':31})

In [34]:
# Train valid split
emocause_valid_shuffled = df_clean[:3775].sample(frac=1)
emocause_train_preprocessed = emocause_valid_shuffled[:3020]
emocause_valid_preprocessed = emocause_valid_shuffled[3020:]

In [35]:
# Test split
emocause_test_preprocessed = df_clean[3775:]

In [36]:
from google.colab import files
emocause_train_preprocessed.to_csv('emocause_train_preprocessed.csv')
files.download('emocause_train_preprocessed.csv')
emocause_valid_preprocessed.to_csv('emocause_valid_preprocessed.csv')
files.download('emocause_valid_preprocessed.csv')
emocause_test_preprocessed.to_csv('emocause_test_preprocessed.csv')
files.download('emocause_test_preprocessed.csv')

"\nfrom google.colab import files\nemocause_train_preprocessed.to_csv('emocause_train_preprocessed.csv')\nfiles.download('emocause_train_preprocessed.csv')\nemocause_valid_preprocessed.to_csv('emocause_valid_preprocessed.csv')\nfiles.download('emocause_valid_preprocessed.csv')\nemocause_test_preprocessed.to_csv('emocause_test_preprocessed.csv')\nfiles.download('emocause_test_preprocessed.csv')\n"